In [ ]:
df = pd.read_csv('big_stat.csv')

In [ ]:
min_stats = defaultdict(dict)


In [ ]:
for row in df[(df.DraftPick == 2) & (df.Age <= 22)].itertuples():
    min_stats[row[2]][row[5]] = row

In [ ]:
rows = []
for name, val in min_stats.items():
    miny = min(val.keys())
    rows.append(val[miny])

In [ ]:
np.round(np.array([_[-17:] for _ in rows]).mean(0)).astype(np.int)

In [ ]:
s = 'Ovr=74, Pot=83, Hgt=57, Str=65, Spd=84, Jmp=82, End=50, Ins=50, Dnk=51, _69=73, _70=62, _71=67, oIQ=59, dIQ=60, Drb=88, Pss=75, Reb=45'

In [ ]:
len(min_stats)

In [ ]:
print(','.join([_.split('=')[0] for _ in s.split(', ')]))

In [ ]:
import json
import pickle
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dfs = []
for i in range(1990,2020):
    dfs.append(np.load('tables_{}.pkl'.format(i)))

In [ ]:
pv = defaultdict(list)
initial_list = set()
for idx,df in enumerate(dfs):
    for team in df.values():
        for row in team['advanced'].itertuples(): #advanced
            x = row[3]
            x = team['totals'].loc[row[0]][2]
            pv[row[0]].append((x,row[-6])) #-6 # -11
            if idx == 0:
                initial_list.add(row[0])

In [ ]:
pv['Stephen Curry']


In [ ]:
data = []
cntr = 0
for player,v in pv.items():
    if player in initial_list:
        #print(player)
        continue
    try:
        #print(v)
        c = False
        for j in range(1,min(len(v),20)):
            mp = sum([v[k][0] for k in range(0,j)])
            #print(v,j)
            data.append((mp,v[j][1]-v[j-1][1]))
            c = True
        cntr += int(c)
    except:
        pass
cntr, len(initial_list)

In [ ]:
data = np.array(data)
#data[:,1]*=100
#data = data[np.where(data[:,0] > 100)]
data.shape

In [ ]:
import statsmodels.api as sm

In [ ]:
res = sm.RLM(data[:,1],sm.add_constant(data[:,0])).fit()
res.summary()

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(data[:,0],data[:,1],s=4)
x2 = sm.add_constant(np.linspace(0,data.max(0)[0]))
y2 = res.predict(x2)
plt.plot(x2,y2,c='k')
#plt.ylim(-10,10)

plt.xlabel('minutes played')
plt.ylabel('3Pr improvement next season')
plt.title('minutes played versus improvement for NBA 2000-2019 (first year of career)')
plt.savefig('mpib2.png',facecolor='w',edgecolor='w')
